In [11]:
import cv2
import numpy as np
import time
from playsound import playsound
import tensorflow as tf
# Load the Haar cascade for face detection
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

path_to_model = r"C:\Users\pc\Dropbox\My PC (DESKTOP-FQDJ4K7)\Desktop\Model\cnn_model.h5"
new_model = tf.keras.models.load_model(path_to_model)
cap = cv2.VideoCapture(0) # Capture through Webcam

# Check if the webcam is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open Webcam")

closed_eye_count = 0
start_time = time.time()
alert_duration = 2  

# Sound file path for the alert
sound_file_path = r"C:\Users\pc\Dropbox\My PC (DESKTOP-FQDJ4K7)\Desktop\Model\emergency-alarm-with-reverb-29431 (1).mp3"

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
   
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) == 0:
            
            closed_eye_count = 0
            start_time = time.time()
            
        for (ex, ey, ew, eh) in eyes:
            eyes_roi = roi_gray[ey:ey+eh, ex:ex+ew]
            eyes_roi_resized = cv2.resize(eyes_roi, (24, 24))
            eyes_roi_resized = np.expand_dims(eyes_roi_resized, axis=-1)  
            eyes_roi_resized = np.expand_dims(eyes_roi_resized, axis=0)  
            eyes_roi_resized = eyes_roi_resized / 255.0  
            
           
            Predictions = new_model.predict(eyes_roi_resized)
            predicted_probability = Predictions[0][0] 
            
            if predicted_probability > 1:
                status = "Closed Eye"
            else:
                status = "Open Eye"
                closed_eye_count = 0
                start_time = time.time()
            
           
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            
            # Add status text
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(roi_color, status, (ex, ey-10), font, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
    #status = "Open Eye"
    elapsed_time = time.time() - start_time
    if status == "Closed Eye":
        closed_eye_count += 1
        if closed_eye_count >= alert_duration * 4:  
            print("Alert: Eyes closed for more than {:.1f} seconds!".format(alert_duration))
            # Perform the sound alert
            playsound(sound_file_path)
            # Reset the closed eye count
            closed_eye_count = 0
    
    
    cv2.imshow('Drowsiness Detection', frame)
    
    if cv2.waitKey(250) & 0xFF == ord('q'):  
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 34ms/step


KeyboardInterrupt: 

In [3]:
import os

current_directory = os.getcwd()
print("Current working directory:", current_directory)

new_directory = r'C:\Users\pc\Dropbox\My PC (DESKTOP-FQDJ4K7)\Desktop\Model'

os.chdir(new_directory)

# Verify the change in working directory
current_directory = os.getcwd()
print("New working directory:", current_directory)

Current working directory: C:\Users\pc
New working directory: C:\Users\pc\Dropbox\My PC (DESKTOP-FQDJ4K7)\Desktop\Model


In [4]:
pip install playsound

Note: you may need to restart the kernel to use updated packages.


In [10]:
import cv2
import numpy as np
import time
import tensorflow as tf
from playsound import playsound


faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

path_to_model = r"C:\Users\pc\Dropbox\My PC (DESKTOP-FQDJ4K7)\Desktop\Model\Transfer_Learning_Model.h5"
new_model = tf.keras.models.load_model(path_to_model)

cap = cv2.VideoCapture(0) # Capture through Webcam

# Check if the webcam is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open Webcam")

closed_eye_count = 0
start_time = time.time()
alert_duration = 2  # Time in seconds for the alert to trigger

sound_file_path = "C:/Users/pc/Dropbox/My PC (DESKTOP-FQDJ4K7)/Desktop/Model/emergency-alarm-with-reverb-29431 (1).mp3"

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale frame
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        # Detect eyes in the face region
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        for (ex, ey, ew, eh) in eyes:
            eyes_roi = roi_gray[ey:ey+eh, ex:ex+ew]
            eyes_roi_resized = cv2.resize(eyes_roi, (224, 224))
            eyes_roi_resized = cv2.cvtColor(eyes_roi_resized, cv2.COLOR_GRAY2RGB)  
            eyes_roi_resized = np.expand_dims(eyes_roi_resized, axis=0)  
            eyes_roi_resized = eyes_roi_resized / 255.0  # Normalize to [0, 1]
            
            # Perform prediction on the eye region
            Predictions = new_model.predict(eyes_roi_resized)
            predicted_probability = Predictions[0][0] 
            
            if predicted_probability < 1:
                status = "Closed Eye"
            else:
                status = "Open Eye"
            
            # Draw rectangle around eyes
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            
            # Add status text
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(roi_color, status, (ex, ey-10), font, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
            
            # Check if eyes are closed for more than the alert duration
            elapsed_time = time.time() - start_time
            if status == "Closed Eye":
                closed_eye_count += 1
                if closed_eye_count >= alert_duration * 5: 
                    print("Alert: Eyes closed for more than {:.1f} seconds!".format(alert_duration))
                    
                    playsound(sound_file_path)
                    # Reset the closed eye count
                    closed_eye_count = 0
    
    # Display the frame with annotations
    cv2.imshow('Drowsiness Detection', frame)
    
    if cv2.waitKey(200) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 86ms/step



    Error 259 for command:
        play "C:\Users\pc\AppData\Local\Temp\PSshsmdrmu.mp3" wait
    The driver cannot recognize the specified command parameter.

    Error 305 for command:
        close "C:\Users\pc\AppData\Local\Temp\PSshsmdrmu.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.
Failed to close the file: "C:\Users\pc\AppData\Local\Temp\PSshsmdrmu.mp3"


Alert: Eyes closed for more than 2.0 seconds!


PlaysoundException: 
    Error 259 for command:
        play "C:\Users\pc\AppData\Local\Temp\PSshsmdrmu.mp3" wait
    The driver cannot recognize the specified command parameter.